In [3]:
import os
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model

# Load MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)

# Path to test images
base_path = os.path.join('..', 'Data', 'test')
output_path = os.path.join('features', 'features_test.csv')

def preprocess_image(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Image not loaded correctly.")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)
        return image
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

# Extract features
test_features = []
for fname in os.listdir(base_path):
    if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(base_path, fname)
        image = preprocess_image(img_path)
        if image is not None:
            image = np.expand_dims(image, axis=0)
            feature_vector = model.predict(image, verbose=0).flatten()
            test_features.append({
                'file': fname,
                **{f'feat_{i}': float(v) for i, v in enumerate(feature_vector)}
            })

# Save test features
df_test = pd.DataFrame(test_features)
df_test.to_csv(output_path, index=False)
print(f"✅ Saved test features to {output_path} with {len(df_test)} images.")


✅ Saved test features to features\features_test.csv with 100 images.
